In [17]:
!pip install streamlit ngrok

In [2]:
!pip install pyngrok
!pip install uvicorn==0.15.0
!pip install streamlit ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.2/732.2 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-7.0.2-py3-none-any.whl size=21233 sha256=15a9e5304a484851910558e176f1dd04e8e27fa64911ba7075d6a23ee0981eb8
  Stored in directory: /root/.cache/pip/wheels/63/3f/ca/3ee63fa3bf9dfcf6014bb5ea56026c8b218ad26b422fd9cce1
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.


In [3]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [34]:
%%writefile app.py
import streamlit as st
import requests
from PIL import Image
import io
import base64
import streamlit as st
from PIL import Image
import json

def base64_to_image(base64_str):
  img_data = base64.b64decode(base64_str)
  return Image.open(io.BytesIO(img_data))




st.title('MEDICAL IMAGES SEGMENTATION WITH DEEP LEARNING MODELS')
st.markdown('---')
st.write('In this application, the inference of 4 deep learning models is carried out: [*UNET*](https://arxiv.org/pdf/1505.04597.pdf), [*MF2-Net*](https://www.sciencedirect.com/science/article/abs/pii/S0952197622001919), [*SAM*](https://arxiv.org/pdf/2304.02643.pdf) and [*SAM-UNETR*](https://ieeexplore.ieee.org/abstract/document/10292632). Training the first two models and fine-tuning the last two models on your decoder block.')
st.markdown('---')
uploaded_file = st.file_uploader("Select the image to upload", type=['png', 'jpg', 'jpeg'])

if uploaded_file is not None:
    # Para leer la imagen
    image = Image.open(uploaded_file)

    # Muestra la imagen
    st.image(image, caption='Image',width=250)
    st.markdown('---')

    buffer = io.BytesIO()
    image.save(buffer, format="JPEG")
    buffer.seek(0)
    img_bytes = buffer.getvalue()
    img_base64 = base64.b64encode(img_bytes)
    img_base64_str = img_base64.decode('utf-8')



    url= 'https://75c3-34-143-149-107.ngrok-free.app/predict/'

    headers = {
    "accept": "application/json",
    "Content-Type": "application/json"}

    data = {
      'img': img_base64_str,
      'format': 'jpg'
      }

    response = requests.post(url,headers=headers, data=json.dumps(data))

    data = response.json()


    mask1 = base64_to_image(data['mask1'])
    mask2 = base64_to_image(data['mask2'])
    mask3 = base64_to_image(data['mask3'])
    mask4 = base64_to_image(data['mask4'])
    mask1 = mask1.convert('L')
    mask2 = mask2.convert('L')
    mask3 = mask1.convert('L')
    mask4 = mask1.convert('L')



    st.markdown("<h2 style='text-align: center; color: black;'>Models inference</h2>", unsafe_allow_html=True)
    #img1,img2,img3,img4 = image
    col1, col2 = st.columns(2)
    with col1:
      st.image(mask1, caption='UNET')
    with col2:
      st.image(mask2, caption='MF2-Net')

    col3, col4 = st.columns(2)
    with col3:
      st.image(mask3, caption='SAM')
    with col4:
      st.image(mask4, caption='SAM-UNETR')

st.markdown('---')

Overwriting app.py


In [19]:
!nohup streamlit run app.py &

nohup: appending output to 'nohup.out'


In [20]:
from pyngrok import ngrok

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken (optional)
# Get your authtoken from https://dashboard.ngrok.com/auth
ngrok.set_auth_token("2YxIuzp5EMvBEYeuwyff1Kdja47_33e4utMDiRySLB7JTERKc")

# Open a HTTP tunnel on the port 8501 (default port)
public_url = ngrok.connect('8501')
public_url

<NgrokTunnel: "https://195c-35-196-229-31.ngrok-free.app" -> "http://localhost:8501">